In [2]:
import string

class CombinedCipher:

    # Combined Vigenère + Affine Cipher

    def __init__(self):
        self.alphabet = string.ascii_uppercase
        self.M = 26

    def _gcd(self, a, b):
        while b:
            a, b = b, a % b
        return a

    def _prepare_text(self, text):
        return ''.join(c.upper() for c in text if c.isalpha())

    # Affine Implementation

    def affine_encrypt(self, plaintext, a, b):
        #Encrypting using Affine cipher: C = (aX + b) mod 26
        if self._gcd(a, self.M) != 1:
            raise ValueError(f"'a' must be coprime with {self.M}. Got a={a}")

        plaintext = self._prepare_text(plaintext)
        ciphertext = []
        for char in plaintext:
            p = self.alphabet.index(char)
            c = (a * p + b) % self.M
            ciphertext.append(self.alphabet[c])
        return ''.join(ciphertext)

    def affine_decrypt(self, ciphertext, a, b):
        #Decrypting using Affine cipher: P = (a^-1 * (Y - b)) mod 26
        if self._gcd(a, self.M) != 1:
            raise ValueError(f"'a' must be coprime with {self.M}. Got a={a}")

        ciphertext = self._prepare_text(ciphertext)
        a_inv = pow(a, -1, self.M)
        plaintext = []
        for char in ciphertext:
            c = self.alphabet.index(char)
            p = (a_inv * (c - b)) % self.M
            plaintext.append(self.alphabet[p])
        return ''.join(plaintext)

    # Vigenère Implementation

    def vigenere_encrypt(self, plaintext, key):
        # Encrypting using Vigenère cipher: C = (P + K[i]) mod 26
        plaintext = self._prepare_text(plaintext)
        key = self._prepare_text(key)
        ciphertext = []
        key_index = 0

        for char in plaintext:
            p = self.alphabet.index(char)
            k = self.alphabet.index(key[key_index % len(key)])
            c = (p + k) % self.M
            ciphertext.append(self.alphabet[c])
            key_index += 1
        return ''.join(ciphertext)

    def vigenere_decrypt(self, ciphertext, key):
        #Decrypting using Vigenère cipher: P = (C - K[i]) mod 26
        ciphertext = self._prepare_text(ciphertext)
        key = self._prepare_text(key)
        plaintext = []
        key_index = 0

        for char in ciphertext:
            c = self.alphabet.index(char)
            k = self.alphabet.index(key[key_index % len(key)])
            p = (c - k) % self.M
            plaintext.append(self.alphabet[p])
            key_index += 1
        return ''.join(plaintext)

    #  Combined Functions

    def combined_encrypt(self, plaintext, vigenere_key, affine_a, affine_b, order="VA"):

        #Combined encryption: Vigenère + Affine (VA) or Affine + Vigenère (AV).
        if order == "VA":
            temp = self.vigenere_encrypt(plaintext, vigenere_key)
            ciphertext = self.affine_encrypt(temp, affine_a, affine_b)
        else:  # AV
            temp = self.affine_encrypt(plaintext, affine_a, affine_b)
            ciphertext = self.vigenere_encrypt(temp, vigenere_key)
        return ciphertext

    def combined_decrypt(self, ciphertext, vigenere_key, affine_a, affine_b, order="VA"):

        #Combined Decryption: Reverses the encryption order.
        if order == "VA":
            # Decrypt: Affine inverse -> Vigenère inverse
            temp = self.affine_decrypt(ciphertext, affine_a, affine_b)
            plaintext = self.vigenere_decrypt(temp, vigenere_key)
        else:  # AV
            # Decrypt: Vigenère inverse -> Affine inverse
            temp = self.vigenere_decrypt(ciphertext, vigenere_key)
            plaintext = self.affine_decrypt(temp, affine_a, affine_b)
        return plaintext


In [3]:
cipher = CombinedCipher()

# text and keys for testing
plaintext = "THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG"
vigenere_key = "KEYKEYKEY"
affine_a = 5
affine_b = 8

# Encrypt and Decrypt using Combined Cipher (VA order)
ciphertext = cipher.combined_encrypt(plaintext, vigenere_key, affine_a, affine_b, order="VA")
decrypted_text = cipher.combined_decrypt(ciphertext, vigenere_key, affine_a, affine_b, order="VA")

print("Plaintext: ", plaintext)
print("Ciphertext: ", ciphertext)
print("Decrypted Text: ", decrypted_text)


Plaintext:  THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG
Ciphertext:  XLSIYMQADNUETBQRVUOZKYDSNTHAFYBSNYG
Decrypted Text:  THEQUICKBROWNFOXJUMPSOVERTHELAZYDOG


In [4]:
# CipherBreaker class for frequency attacks

import time
from collections import Counter

class CipherBreaker:

    def __init__(self, encryption_order="VA"):
        self.encryption_order = encryption_order
        self.alphabet = string.ascii_uppercase
        self.M = 26
        self.attack_log = []

        # English letter frequencies (in percentages)
        self.ENGLISH_FREQ = {
            'E': 12.70, 'T': 9.06, 'A': 8.17, 'O': 7.51, 'I': 6.97,
            'N': 6.75, 'S': 6.33, 'H': 6.09, 'R': 5.99, 'D': 4.25,
            'L': 4.03, 'C': 2.78, 'U': 2.76, 'M': 2.41, 'W': 2.36,
            'F': 2.23, 'G': 2.02, 'Y': 1.97, 'P': 1.93, 'B': 1.29,
            'V': 0.98, 'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10, 'Z': 0.07
        }

    def log(self, message):
        self.attack_log.append(message)
        print(message)

    def clear_log(self):
        self.attack_log = []

    def frequency_analysis(self, text):
        text = text.upper()
        total = sum(1 for c in text if c.isalpha())
        if total == 0: return {}
        freq = Counter(c for c in text if c.isalpha())
        return {letter: (count / total) * 100 for letter, count in freq.items()}

    def chi_squared(self, observed_freq):
        chi_sq = 0
        for letter in self.alphabet:
            expected = self.ENGLISH_FREQ.get(letter, 0)
            observed = observed_freq.get(letter, 0)
            if expected > 0:
                chi_sq += ((observed - expected) ** 2) / expected
        return chi_sq

    def index_of_coincidence(self, text):
        """Calculate Index of Coincidence for the text."""
        text = ''.join(c for c in text.upper() if c.isalpha())
        n = len(text)
        if n <= 1: return 0
        freq = Counter(text)
        ic = sum(f * (f - 1) for f in freq.values()) / (n * (n - 1))
        return ic

    def estimate_key_length(self, ciphertext, max_length=20):
        #Estimate Vigenère key length using Index of Coincidence.
        self.log("\nESTIMATING VIGENÈRE KEY LENGTH (IC Analysis)")
        ciphertext = ''.join(c for c in ciphertext.upper() if c.isalpha())
        ic_scores = {}
        for key_len in range(1, max_length + 1):
            groups = [''] * key_len
            for i, char in enumerate(ciphertext):
                groups[i % key_len] += char
            avg_ic = sum(self.index_of_coincidence(g) for g in groups) / key_len
            ic_scores[key_len] = avg_ic

        sorted_keys = sorted(ic_scores.items(), key=lambda x: abs(x[1] - 0.067))
        best_key_length = sorted_keys[0][0]
        self.log(f"Most likely key length: **{best_key_length}** (IC = {sorted_keys[0][1]:.4f})")
        return best_key_length, ic_scores


In [5]:
breaker = CipherBreaker(encryption_order="VA")

ciphertext = 'QEB NRFZH YOLTK CLU GRJMP LSBO QEB IXWV ALD'

freq_result = breaker.frequency_analysis(ciphertext)
print("Frequency Analysis: ", freq_result)

key_length, ic_scores = breaker.estimate_key_length(ciphertext)
print("Estimated Key Length: ", key_length)


Frequency Analysis:  {'Q': 5.714285714285714, 'E': 5.714285714285714, 'B': 8.571428571428571, 'N': 2.857142857142857, 'R': 5.714285714285714, 'F': 2.857142857142857, 'Z': 2.857142857142857, 'H': 2.857142857142857, 'Y': 2.857142857142857, 'O': 5.714285714285714, 'L': 11.428571428571429, 'T': 2.857142857142857, 'K': 2.857142857142857, 'C': 2.857142857142857, 'U': 2.857142857142857, 'G': 2.857142857142857, 'J': 2.857142857142857, 'M': 2.857142857142857, 'P': 2.857142857142857, 'S': 2.857142857142857, 'I': 2.857142857142857, 'X': 2.857142857142857, 'W': 2.857142857142857, 'V': 2.857142857142857, 'A': 2.857142857142857, 'D': 2.857142857142857}

ESTIMATING VIGENÈRE KEY LENGTH (IC Analysis)
Most likely key length: **15** (IC = 0.0667)
Estimated Key Length:  15


In [6]:
def shift_cipher_encrypt(plaintext, shift):
    result = []
    for char in plaintext:
        if char.isalpha():
            result.append(chr(((ord(char.upper()) - 65 + shift) % 26) + 65))
        else:
            result.append(char)
    return ''.join(result)

def shift_cipher_decrypt(ciphertext, shift):
    return shift_cipher_encrypt(ciphertext, -shift)


In [8]:
import time

def run_tests():
    cipher = CombinedCipher()

    # Test configurations with varying plaintext lengths
    test_cases = [
        {
            'name': 'Test Case 1: VA Order (Vigenère -> Affine) - Short Length',
            'plaintext': 'HELLO',
            'vigenere_key': 'SECRET',
            'affine_a': 5,
            'affine_b': 8,
            'order': 'VA'
        },
        {
            'name': 'Test Case 2: VA Order (Vigenère -> Affine) - Medium Length',
            'plaintext': 'THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG',
            'vigenere_key': 'CRYPTOGRAPHYSECRET',
            'affine_a': 5,
            'affine_b': 8,
            'order': 'VA'
        },
        {
            'name': 'Test Case 3: AV Order (Affine -> Vigenère) - Medium Length',
            'plaintext': 'NETWORK AND INFORMATION SECURITY IS A CRITICAL FIELD',
            'vigenere_key': 'PROJECTNIS486KEY',
            'affine_a': 9,
            'affine_b': 15,
            'order': 'AV'
        },
        {
            'name': 'Test Case 4: VA Order (Vigenère -> Affine) - Long Length',
            'plaintext': 'A' * 1000,  # 1000 character string
            'vigenere_key': 'LONGKEYLONGKEY',
            'affine_a': 7,
            'affine_b': 3,
            'order': 'VA'
        },
        {
            'name': 'Test Case 5: AV Order (Affine -> Vigenère) - Long Length',
            'plaintext': 'B' * 1000,  # 1000 character string
            'vigenere_key': 'LONGKEYLONGKEY',
            'affine_a': 11,
            'affine_b': 20,
            'order': 'AV'
        }
    ]

    # Shift Cipher test configuration
    shift_key = 3

    results_summary = []

    # Test for Combined Cipher
    for test in test_cases:
        total_time = 0
        iterations = 10

        for _ in range(iterations):
            start_time = time.time()

            # Encrypt the plaintext
            ciphertext = cipher.combined_encrypt(test['plaintext'], test['vigenere_key'], test['affine_a'], test['affine_b'], test['order'])

            # Decrypt the ciphertext
            decrypted_text = cipher.combined_decrypt(ciphertext, test['vigenere_key'], test['affine_a'], test['affine_b'], test['order'])

            # Success Check (Comparing decrypted text to the original plaintext)
            success = decrypted_text.replace(" ", "") == test['plaintext'].replace(" ", "")

            elapsed_time = time.time() - start_time
            total_time += elapsed_time

        avg_time = total_time / iterations
        results_summary.append({
            'test_name': test['name'],
            'success': '✓ PASS' if success else '✗ FAIL',
            'avg_time_taken': avg_time,
            'plaintext_length': len(test['plaintext'])
        })

        print(f"{test['name']} - {'✓ PASS' if success else '✗ FAIL'} | Average time taken: {avg_time:.6f} seconds | Plaintext Length: {len(test['plaintext'])}")

    # Test for Shift Cipher (Caesar Cipher)
    for test in test_cases:
        total_time = 0
        iterations = 10

        for _ in range(iterations):
            start_time = time.time()

            # Encrypt with Shift Cipher
            shift_ciphertext = shift_cipher_encrypt(test['plaintext'], shift_key)
            # Decrypt with Shift Cipher
            shift_decrypted_text = shift_cipher_decrypt(shift_ciphertext, shift_key)

            # Success Check (Comparing decrypted text to the original plaintext)
            shift_success = shift_decrypted_text.replace(" ", "") == test['plaintext'].replace(" ", "")

            shift_elapsed_time = time.time() - start_time
            total_time += shift_elapsed_time

        avg_shift_time = total_time / iterations
        results_summary.append({
            'test_name': f"Shift Cipher - {test['name']}",
            'success': '✓ PASS' if shift_success else '✗ FAIL',
            'avg_time_taken': avg_shift_time,
            'plaintext_length': len(test['plaintext'])
        })

        print(f"Shift Cipher - {test['name']} - {'✓ PASS' if shift_success else '✗ FAIL'} | Average time taken: {avg_shift_time:.6f} seconds | Plaintext Length: {len(test['plaintext'])}")

    return results_summary

test_results = run_tests()


Test Case 1: VA Order (Vigenère -> Affine) - Short Length - ✓ PASS | Average time taken: 0.000018 seconds | Plaintext Length: 5
Test Case 2: VA Order (Vigenère -> Affine) - Medium Length - ✓ PASS | Average time taken: 0.000106 seconds | Plaintext Length: 43
Test Case 3: AV Order (Affine -> Vigenère) - Medium Length - ✓ PASS | Average time taken: 0.000091 seconds | Plaintext Length: 52
Test Case 4: VA Order (Vigenère -> Affine) - Long Length - ✓ PASS | Average time taken: 0.001766 seconds | Plaintext Length: 1000
Test Case 5: AV Order (Affine -> Vigenère) - Long Length - ✓ PASS | Average time taken: 0.001810 seconds | Plaintext Length: 1000
Shift Cipher - Test Case 1: VA Order (Vigenère -> Affine) - Short Length - ✓ PASS | Average time taken: 0.000006 seconds | Plaintext Length: 5
Shift Cipher - Test Case 2: VA Order (Vigenère -> Affine) - Medium Length - ✓ PASS | Average time taken: 0.000026 seconds | Plaintext Length: 43
Shift Cipher - Test Case 3: AV Order (Affine -> Vigenère) - Medi